# LSTM Statistic Predictor Using OPS(On-base Plus Slugging)

In this notebook, we will build and train a custom LSTM RNN that uses a 12 years of batting statistics data to predict the value of OPS.

## Statistics Use In Moneyball

According to Lewis (2003), Billy Beane (the inspiration of Moneyball) decided to base his drafting of position players/hitters on certain statistics. His main two statistics included `on-base percentage (OBP)` and `slugging percentage`. These two stats combined to form a new statistic called `on-base plus slugging (OPS)`.

The OPS formula below:

        OBP = (H + BB + HBP) / (AB + BB + HBP + SF)
        
        SLG = (1B)+(2∗2B)+(3∗3B)+(4∗HR) / AB
        

H = Hits - when the batter strikes the ball without error

BB = Walks - when a pitcher throws four pitches out of the strike zone, none of which are swing at by the batter

HBP = Hit by pitch - when a batter is struck by a pitched ball without swinging at it and is awarded first base

AB = At bat - when a batter reaches base via fielder's choice, hit, or error (not including catcher's interference)

SF = Sacrifice fly - when a batter hits a fly-ball to the outfield or foul territory that allows a runner to score

1B = Single - when batter hits the ball and reaches first base

2B = Double - when batter hits the ball and reaches second base

3B = Triple - when batter hits the ball and reaches third base

HR = Home run - when batter hits the ball and circles all four bases

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from pathlib import Path
import glob

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Denssorflow as tfe
import ten
import warnings
warnings.filterwarnisorflw as tf


%m
atplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\

In [3]:
# Import dataset
path = "Resources/batting_stats_*.csv"
all_files = glob.glob(path)
all_files

# Read & combine into dataframe
data = []

for file in all_files:
    df = pd.read_csv(file, index_col=None, header=0)    
    year = file[-8:-4]
    df['Year'] = year
    data.append(df)

stats_df = pd.concat(data, axis=0, ignore_index=True)
stats_df = stats_df[['Year', 'Player', 'Team', 'Pos', 'Age', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR','RBI', 
             'SB', 'CS', 'BB', 'SO', 'SH', 'SF', 'HBP', 'AVG', 'OBP', 'SLG','OPS']]

# Drop player with 0 OPS
stats_df = stats_df[stats_df.OPS !=0].set_index('Player')

stats_df.head(100)

,Year,Team,Pos,Age,G,AB,R,H,2B,3B,...,CS,BB,SO,SH,SF,HBP,AVG,OBP,SLG,OPS
Player,,,,,,,,,,,,,,,,,,,,,
Ichiro Suzuki,2010,SEA,OF,47,162,680,74,214,30,3,...,9,45,86,3,1,3,0.315,0.359,0.394,0.753
Derek Jeter,2010,NYY,SS,47,157,663,111,179,30,3,...,5,63,106,1,3,9,0.270,0.340,0.370,0.710
Michael Young,2010,TEX,3B,44,157,656,99,186,36,3,...,2,50,115,0,11,1,0.284,0.330,0.444,0.774
Juan Pierre,2010,CWS,OF,43,160,651,96,179,18,3,...,18,45,47,15,2,21,0.275,0.341,0.316,0.657
Rickie Weeks,2010,MIL,DH,38,160,651,112,175,32,4,...,4,76,184,0,2,25,0.269,0.366,0.464,0.830
Marco Scutaro,2010,BOS,2B,45,150,632,92,174,38,0,...,4,53,71,4,3,3,0.275,0.333,0.388,0.721
Nick Markakis,2010,BAL,OF,37,160,629,79,187,45,3,...,2,73,93,0,5,2,0.297,0.370,0.436,0.806
Denard Span,2010,MIN,OF,37,153,629,85,166,24,10,...,4,60,74,10,2,4,0.264,0.331,0.348,0.679
Brandon Phillips,2010,CIN,2B,40,155,626,100,172,33,5,...,12,46,83,6,1,8,0.275,0.332,0.430,0.762


In [4]:
# Creating X & y variables
X = stats_df.iloc[:, 4:-2]
y = stats_df["OPS"].values

In [5]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train.shape

(5914, 17)

In [6]:
# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

y = y.reshape(-1, 1)
scaler_y = StandardScaler().fit(y)
y = scaler_y.transform(y)

In [7]:
y_train.shape

(5914,)

In [8]:
#X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
#X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [9]:
# Model set-up
number_input_features = 17
hidden_nodes_layer1 = 34
hidden_nodes_layer2 = 5

In [10]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1
model.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Layer 2
model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
model.add(Dense(1, activation="sigmoid"))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# Training the model
batch_size = 1000
epochs = 100
training_history = model.fit(
    X_train,
    y_train,
    #validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

Epoch 1/100
5914/5914 [==============================] - 0s 23us/sample - loss: 0.6718 - acc: 0.0024 - tp: 5505.0000 - tn: 0.0000e+00 - fp: 0.0000e+00 - fn: 409.0000 - precision: 1.0000 - recall: 0.9308 - auc: 0.0000e+00
Epoch 2/100
5914/5914 [==============================] - 0s 2us/sample - loss: 0.6422 - acc: 0.0030 - tp: 5901.0000 - tn: 0.0000e+00 - fp: 0.0000e+00 - fn: 13.0000 - precision: 1.0000 - recall: 0.9978 - auc: 0.0000e+00
Epoch 3/100
5914/5914 [==============================] - 0s 2us/sample - loss: 0.6317 - acc: 0.0030 - tp: 5908.0000 - tn: 0.0000e+00 - fp: 0.0000e+00 - fn: 6.0000 - precision: 1.0000 - recall: 0.9990 - auc: 0.0000e+00
Epoch 4/100
5914/5914 [==============================] - 0s 2us/sample - loss: 0.6227 - acc: 0.0030 - tp: 5911.0000 - tn: 0.0000e+00 - fp: 0.0000e+00 - fn: 3.0000 - precision: 1.0000 - recall: 0.9995 - auc: 0.0000e+00
Epoch 5/100
5914/5914 [==============================] - 0s 3us/sample - loss: 0.6167 - acc: 0.0034 - tp: 5913.0000 - tn: 0.

In [13]:
# Make predictions using the testing data X_test
predicted = model.predict(X_test)

# Evaluate the model
model.evaluate(X_test, y_test)

1972/1972 [==============================] - 0s 43us/sample - loss: 0.5085 - acc: 0.0056 - tp: 1814.0000 - tn: 0.0000e+00 - fp: 0.0000e+00 - fn: 158.0000 - precision: 1.0000 - recall: 0.9199 - auc: 0.0000e+00


[0.5084630828841946, 0.005578093, 1814.0, 0.0, 0.0, 158.0, 1.0, 0.9198783, 0.0]

In [14]:
predicted_OPS = scaler_y.inverse_transform(predicted)

In [15]:
moneyball = pd.DataFrame({
    "Real": y_test.ravel(),
    "Predicted": predicted_OPS.ravel()
    }, index = stats_df.index[-len(predicted_OPS): ])
moneyball.head(20)

,Real,Predicted
Player,,
Jacob Nottingham,0.950,0.852732
Taylor Motter,0.651,0.786296
Ildemaro Vargas,0.616,0.769089
Adam Rosales,0.614,0.783100
Brandon Barnes,0.661,0.830018
Anthony Alford,0.646,0.795635
Luis Sardinas,1.185,0.858469
Matt den Dekker,0.419,0.763675
Corban Joseph,0.486,0.768718


In [16]:
moneyball = moneyball.groupby('Player').mean()
moneyball.head(20)

,Real,Predicted
Player,,
AJ Pollock,0.642333,0.784162
AJ Reed,0.696000,0.807914
Aaron Altherr,0.757000,0.823085
Aaron Hicks,0.586000,0.775682
Aaron Judge,0.637000,0.791791
Abiatal Avelino,0.546000,0.784224
Abraham Almonte,0.637333,0.794459
Abraham Toro,0.802667,0.822177
Adalberto Mondesi,0.845333,0.831775


In [17]:
moneyball.to_csv(r'Resources/Result/Moneyball.csv')